# Machine Learning Pipeline with KubeDirector - Lab 1
## Authenticate to Kubernetes cluster deployed on HPE Ezmeral Runtime Enterprise and fetch the kubeconfig file for your tenant

### **Lab workflow**

In this lab:

1. As tenant user, you will first establish a valid login session with HPE Ezmeral Runtime Enterprise (formerly known as HPE Ezmeral Container Platform) using a REST API call.

2. You will then fetch the *kubeconfig* file for your tenant working context using a subsequent REST API.
>Note: A kubeconfig file is used to access the Kubernetes cluster for your tenant working context.


**Recommended workshop and blogs about HPE Ezmeral Runtime Enterprise REST API:**
- Workshop: [Intro to the HPE Ezmeral Container Platform REST API](https://hackshack.hpedev.io/workshops) 
- Blog: [HPE Ezmeral Container Platform REST API Part 1 - Authenticating](https://developer.hpe.com/blog/hpe-container-platform-rest-api-part-1-authenticating)
- Blog: [HPE Ezmeral Container Platform REST API Part 2 - Deploying containerized applications](https://developer.hpe.com/blog/hpe-container-platform-rest-api-part-2-deploying-containerized-applicati)


**Definitions:**

- *HPE Ezmeral Runtime Enterprise* is an enterprise-grade container platform designed to deploy both cloud-native and non-cloud-native applications whether they are on-premises, at the edge, in multiple public clouds, or in a hybrid model. This makes the HPE Ezmeral Runtime Enterprise ideal for helping enterprise customers accelerate their application development and deployment via **containers** on-demand through a self-service portal and a RESTful API that surfaces programmable access. To learn more about HPE Ezmeral Runtime Enterprise, visit the [HPE DEV portal](https://developer.hpe.com/platform/hpe-ezmeral/home) and check out the blog articles.

- *Tenant:* A tenant is a group of users created by the platform administrator. A tenant can represent, for example, an office location, a business unit, an organization, or a project. A tenant is allocated a quota of resources (CPU, GPU, memory, storage, and Kubernetes clusters resources) by the platform administrator. All the resources used by a tenant are not shared with other tenants. The tenant serves as a Kubernetes namespace.

- *Working tenant context:* A working tenant context establishes the user identity, its tenant name and tenant role (member or admin). Based on this context, tenant users are granted privileges and permissions to create and manage resources for their tenant on Kubernetes clusters managed by HPE ECP.

### **1- Initialize the environment**

Let's first define the environment variables according to your HPE Ezmeral Runtime Enterprise user account and tenant name, and the HPE Ezmeral Runtime Enterprise API system endpoint.

> **Note:** When executing a notebook code cell, a [*] next to the action, it means the execution step is busy working within the notebook.   
A [digit number] next to the action means the execution step has completed.

In [ ]:
#
# Environment variables for your working tenant context
#
studentId="student{{ STDID }}" # your Jupyter Notebook student Identifier (i.e.: student<xx>)
username="student{{ STDID }}" # your HPE Ezmeral tenant login credentials - the username is your jupyterHub account username 
password="{{ PASSSTU }}" # your HPE Ezmeral tenant login credentials - the password is your jupyterHub account password

gateway_host="{{ HPEECPGWNAME }}"
controller_endpoint="${gateway_host}:8080"

tenantname="K8sMLTenant" #case sensitive

echo "your operation context is:" $username "on tenant" $tenantname 
echo "your REST API endpoint is:" $controller_endpoint

### **2- Authenticate as a tenant user in the specified tenant**

In [ ]:
sessionlocation=$(curl -k -i -s --request POST "https://${controller_endpoint}/api/v2/session" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--data-raw '{
"name": "'"$username"'",
"password": "'"$password"'",
"tenant_name": "'"$tenantname"'"
}' | grep Location | awk '{print $2}' | tr -d '\r') #we remove any cr that might exist
echo "This is your session location: " $sessionlocation
SessionId=$(echo $sessionlocation | cut -d'/' -f 5) # extract sessionId for later, for logout
echo "This is your session_Id:" $SessionId

### **3- Get the Kubeconfig file for your tenant working context**
The next step in deploying a containerized application in Kubernetes clusters managed by the HPE Ezmeral Runtime Enterprise is to get the kubeconfig file for your tenant working context. 

The HPE Ezmeral Runtime Enterprise REST API call below allows you to get the **kubeconfig file** used to access the Kubernetes cluster for your tenant user account based on your assigned role (tenant member) with the same result as if you had downloaded it from the HPE Ezmeral Runtime Enterprise UI.

In [ ]:
mkdir -p ~/.kube
ls ~/.kube

In [ ]:
rm -f ~/.kube/config
curl -k -s --request GET "https://${controller_endpoint}/api/v2/k8skubeconfig" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' > ~/.kube/config

echo "Kubeconfig file fetched"

#### Check your working tenant context:

In [ ]:
kubectl config current-context

You can now send Kubernetes API requests using a K8s API client such as **kubectl** to set up your machine learning pipeline using KubeDirector and train, register, and query your model.


**Let's see this in action!**
* [Lab2 - Deploy Local Jupyter Notebook Cluster](2-WKSHP-K8s-ML-Pipeline-Deploy-Local-Notebook.ipynb)